In [ ]:
!pip install folium pandas geopandas matplotlib seaborn


Load the dataset into Colab:

In [ ]:
import geopandas as gpd

# Load wildfire CSV
wildfire_data = gpd.read_file('National_USFS_Fire_Occurrence_Point_(Feature_Layer).csv')

# Display the first few rows
wildfire_data.head()


,X,Y,OBJECTID,GLOBALID,FIREOCCURID,CN,REVDATE,FIRENAME,COMPLEXNAME,FIREYEAR,...,LATDD83,LONGDD83,FIRETYPECATEGORY,POINTTYPE,PERIMEXISTS,FIRERPTQC,DBSOURCEID,DBSOURCEDATE,ACCURACY,SHAPE
0,-106.42777778,39.84611111,231055009,{540C6E70-51FD-4CCC-A1B3-8A6C914E34A3},2B1B9A1B-7828-4688-A14C-4BD7CEE62226,,2023/03/29 11:10:56+00,Elliott Ridge,,2016,...,39.84611,-106.42778,WF,General,N,Yes,0215,2024/09/24 04:01:09+00,24000,
1,-107.93555556,39.37277778,231055010,{ACE77CF8-1281-4A81-BE3D-4D55669CB134},A2454D64-EBB7-4895-81BC-9782B3D1391E,,2023/03/29 11:10:59+00,Battlement Mesa Reservoir,,2016,...,39.37278,-107.93556,WF,General,N,Yes,0215,2024/09/24 04:01:15+00,24000,
2,-106.72333333,39.11833333,231055011,{D8EA7329-A0D0-4B64-A6F2-7A7FD86A3505},,,2023/03/29 11:10:56+00,Difficult,,1997,...,39.11833,-106.72333,WF,General,N,Yes,0215,2024/09/24 04:01:15+00,24000,
3,-106.59,39.65666667,231055012,{67A72390-0E2B-411E-9CDE-6657101645E4},,,2023/03/29 11:10:56+00,,,1993,...,39.65667,-106.59,WF,General,N,Yes,0215,2024/09/24 04:01:15+00,24000,
4,-107.317310669,40.019286292,231055013,{DFFF8D93-DE89-451C-830C-1F65FD550B3D},558D5D61-AB04-4AC3-8143-2E0EBE7CB506,,2024/03/27 14:21:08+00,Paradise,,2020,...,40.01929,-107.31731,WF,General,Y,Yes,0215,2024/09/24 04:01:15+00,24000,


Use Folium to display wildfire locations:

In [ ]:
import folium
import geopandas as gpd
import pandas as pd

# Convert LONGDD83 and LATDD83 to numeric, coercing errors to NaN
wildfire_data['LONGDD83'] = pd.to_numeric(wildfire_data['LONGDD83'], errors='coerce')
wildfire_data['LATDD83'] = pd.to_numeric(wildfire_data['LATDD83'], errors='coerce')

# Convert TOTALACRES to numeric, coercing errors to NaN
wildfire_data['TOTALACRES'] = pd.to_numeric(wildfire_data['TOTALACRES'], errors='coerce')

# Drop rows with invalid coordinates or missing acreage
wildfire_data = wildfire_data.dropna(subset=['LONGDD83', 'LATDD83', 'TOTALACRES'])

# Ensure that the Latitude and Longitude columns are in proper format
wildfire_data['geometry'] = gpd.points_from_xy(wildfire_data['LONGDD83'], wildfire_data['LATDD83'])
wildfire_data = gpd.GeoDataFrame(wildfire_data, geometry='geometry')

# Filter to include only fires within US bounds
us_bounds = wildfire_data[
    (wildfire_data['LATDD83'] >= 24.396308) & (wildfire_data['LATDD83'] <= 49.384358) &
    (wildfire_data['LONGDD83'] >= -125.0) & (wildfire_data['LONGDD83'] <= -66.93457)
]

# Filter the data to include only fires with size greater than 5 acres
filtered_wildfire_data = us_bounds[us_bounds['TOTALACRES'] > 10000]

# Create a base map centered on the US
m = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

# Add fire location markers to the map
for idx, row in filtered_wildfire_data.iterrows():
    fire_info = f"Name: {row['FIRENAME']}<br>Year: {row['FIREYEAR']}<br>Total Acres: {row['TOTALACRES']}"
    folium.CircleMarker(
        location=[row['LATDD83'], row['LONGDD83']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=folium.Tooltip(fire_info, sticky=True)  # Tooltip displays on hover
    ).add_to(m)

# Display the map
m
